### Census Tract Issue - USDA Food Access Research Atlas (Tennessee data only)

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np


Importing both datasets

In [2]:
tn_food_access_df = pd.read_csv('../data/tn_food_access.csv')

In [3]:
tn_food_access_df = tn_food_access_df.drop('State', axis=1)

In [4]:
# I need to see if this is doing something constructive with the data, or anything at all, before running:
# tn_food_access_df = tn_food_access_df.replace(r'^\s*$', np.nan, regex=True)

In [5]:
# tn_food_access_df_fillna.to_csv('tn_food_access_nan.csv')

Getting a general sense of the way census tracts operate in the two datasets ...

In [6]:
tn_food_access_df.head()

,CT_Old,CT_Clean,County,Urban,Pop2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,47001020100,201.00,Anderson County,1,3111,1546,0,6,0.19,0,...,384,1969,853,130,2,9,148,145,206,438
1,47001020201,202.01,Anderson County,1,3670,1535,0,103,2.81,0,...,658,3158,144,256,0,6,106,84,49,31
2,47001020202,202.02,Anderson County,1,4507,2025,0,230,5.10,0,...,1364,3988,194,128,0,18,179,214,174,208
3,47001020300,203.00,Anderson County,1,3822,1656,0,0,0.00,0,...,640,3290,278,20,3,18,213,179,53,109
4,47001020400,204.00,Anderson County,1,4508,2013,0,106,2.35,1,...,761,3855,264,34,3,20,332,277,86,406


In [7]:
tn_food_access_df['CT_Clean'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1489 entries, 0 to 1488
Series name: CT_Clean
Non-Null Count  Dtype  
--------------  -----  
1489 non-null   float64
dtypes: float64(1)
memory usage: 11.8 KB


993 unique entries for census tract, yet 1,489 total? Need to look into this as a top priority.

In [8]:
tn_food_access_df['CT_Clean'].nunique()

993

In [9]:
# https://saturncloud.io/blog/how-to-check-for-duplicate-values-in-pandas-dataframe-column/#:~:text=To%20check%20for%20duplicate%20values%20in%20a%20Pandas%20DataFrame%20column,is%20a%20duplicate%20or%20not.
# duplicate_values = df['Product'].duplicated()
# print(duplicate_values)

duplicate_CTs = tn_food_access_df['CT_Clean'].duplicated()
duplicate_CTs_to_csv = pd.DataFrame(duplicate_CTs)
# duplicate_CTs_to_csv.to_csv('duplicate_CTs.csv')


ISSUE: converting the census tract numbers to match the census reporter data did not work and created duplicate census tract IDs in about 1/3 of the data. I will probably need to go back and change some things but need some help troubleshooting and figuring out where to start/what's needed.

In [10]:
tn_food_access_df['CT_Old'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1489 entries, 0 to 1488
Series name: CT_Old
Non-Null Count  Dtype
--------------  -----
1489 non-null   int64
dtypes: int64(1)
memory usage: 11.8 KB


In [11]:
tn_food_access_df['CT_Old'].nunique()

1489

In [12]:
tn_food_access_df = tn_food_access_df.drop_duplicates()

#### Origin of issue discovery:

Pulling minimum population, I'm seeing two lines that appear to have the same census tract number. 

In [13]:
tn_food_access_df[tn_food_access_df['Pop2010'].iloc[:]==tn_food_access_df['Pop2010'].iloc[:].min()]

,CT_Old,CT_Clean,County,Urban,Pop2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
529,47065980100,801.0,Hamilton County,1,1,1,0,0,0.0,0,...,0,1,0,0,0,0,0,0,0,0
1043,47155980100,801.0,Sevier County,0,1,1,0,0,0.0,0,...,0,1,0,0,0,0,0,0,0,0


After speaking with instructors:
* pull digits only from census_df
* pull out county, create column for that
* make a list out of all column headers
* expand/true on comma
* take CT_Old column from tn_food_access_df and compare
* be sure to add "from income" to census_df and "from food" to tn_food_access_df
* union and catch for duplicates
* see the stuff that only exists in one form or another
* maybe also research census tract numbers (do they come in other forms? why wouldn't they be the same? is one more granular?)

In [14]:
census_df = pd.read_excel('../data/census_tn_income_b19001.xlsx')

KeyboardInterrupt: 

In [ ]:
census_df.head()

In [ ]:
census_df_ct_troubleshoot = census_df.rename(columns={'Unnamed: 0':'income_bracket'})

In [ ]:
census_df_ct_troubleshoot = census_df_ct_troubleshoot[census_df_ct_troubleshoot.columns.drop(list(census_df_ct_troubleshoot.filter(regex='Unnamed')))]


In [ ]:
census_df_ct_troubleshoot.head()

In [ ]:
# this removes the first row of data
census_df_ct_troubleshoot = census_df_ct_troubleshoot.iloc[1:,:]

In [ ]:
census_df_ct_troubleshoot.head()

In [ ]:
census_df_ct_troubleshoot = census_df_ct_troubleshoot.transpose().reset_index()

In [ ]:
census_df_ct_troubleshoot.columns = census_df_ct_troubleshoot.iloc[0]

In [ ]:
census_df_ct_troubleshoot = census_df_ct_troubleshoot.iloc[1:,:]

In [ ]:
census_df_ct_troubleshoot['income_bracket'].replace('Census Tract ','', regex=True, inplace = True)

In [ ]:
# census_df_ct_troubleshoot_test = census_df_ct_troubleshoot['income_bracket'].replace('Census Tract ','', inplace=True)

I've successfully taken "Census Tract" out of the income_bracket column. Now I need to create a new column and use the .split() function to separate the income bracket column into two separate columns: one for the census tract number, and one for the county name. Then I probably want another column (instead of altering the original ct number) which removes the decimal points from the census tracts. 

In [ ]:
census_df_ct_troubleshoot.head()

In [ ]:
census_df_ct_troubleshoot.columns.values.tolist()

In [ ]:
# df[['First Name', 'Last Name']] = df['Name'].str.split(' ', expand=True)
# https://saturncloud.io/blog/how-to-split-one-column-into-multiple-columns-in-pandas-dataframe/

census_df_ct_troubleshoot[['income_bracket','county_cr']] = census_df_ct_troubleshoot['income_bracket'].str.split(',',n=1,expand=True)
census_df_ct_troubleshoot.head()



In [ ]:
census_df_ct_troubleshoot['county_cr'].replace(', TN','', regex=True, inplace = True)

In [ ]:
census_df_ct_troubleshoot.head()

In [ ]:
census_df_ct_troubleshoot = census_df_ct_troubleshoot.rename(columns={'income_bracket':'census_tract','Total:':'total'})

In [ ]:
census_df_ct_troubleshoot.head()

In [ ]:
census_df_ct_troubleshoot['census_tract'].nunique()

In [ ]:
census_df_ct_troubleshoot['ct_digits_cr'] = census_df_ct_troubleshoot['census_tract'].str.replace('.','')

In [ ]:
census_df_ct_troubleshoot.head(200)

In [ ]:
census_df_ct_troubleshoot = census_df_ct_troubleshoot.drop_duplicates()

In [ ]:
census_df_ct_troubleshoot['from'] = 'census'

In [ ]:
census_df_ct_troubleshoot.head()

In [ ]:
# census_df_ct_troubleshoot.to_csv('census_df_latest_2.csv')

In [ ]:
tn_food_access_df.head()

In [ ]:
# census_df_ct_troubleshoot = census_df_ct_troubleshoot.rename(columns={'income_bracket':'census_tract','Total:':'total'})

tn_food_access_df = tn_food_access_df.rename(columns={'CT_Old':'ct_long_usda','CT_Clean':'ct_error_usda','County':'county_usda'})

In [ ]:
tn_food_access_df.head()

In [ ]:
# census_df_ct_troubleshoot['from'] = 'census'
tn_food_access_df['from'] = 'usda'

In [ ]:
tn_food_access_df['county_usda'].replace(' County','', regex=True, inplace = True)

In [ ]:
usda_data_to_union = tn_food_access_df[['ct_long_usda','county_usda','from']].copy()

In [ ]:
tn_food_access_df.head()

In [ ]:
# tn_food_access_df.to_csv('tn_food_latest.csv')

In [ ]:
usda_data_to_union = usda_data_to_union.rename(columns={'ct_long_usda':'ct','county_usda':'county'})

In [ ]:
usda_data_to_union.head()

In [ ]:
cr_data_to_union = census_df_ct_troubleshoot[['ct_digits_cr','county_cr','from']].copy()

In [ ]:
# this removes the first row of data
cr_data_to_union = cr_data_to_union.iloc[1:,:]

In [ ]:
cr_data_to_union = cr_data_to_union.rename(columns={'ct_digits_cr':'ct','county_cr':'county'})

In [ ]:
cr_data_to_union.head()

In [ ]:
usda_data_to_union['ct']=usda_data_to_union['ct'].astype(str)


In [ ]:
usda_data_to_union.dtypes

In [ ]:
cr_data_to_union.dtypes

In [ ]:
# joined_usda_cr_tracts = pd.merge(usda_data_to_union,cr_data_to_union, how='full',on=['county'])# 

In [ ]:
# https://saturncloud.io/blog/how-to-perform-a-union-of-two-pandas-dataframes/
# joined_usda_cr_tracts = pd.concat([usda_data_to_union,cr_data_to_union])

In [ ]:
# https://sql2pandas.pythonanywhere.com/articles/From-SQL-to-pandas-SQL-UNION-ALL-in-pandas
# pd.concat([tips, tips_weekdays], ignore_index=True).drop_duplicates()
joined_usda_cr_tracts = pd.concat([usda_data_to_union,cr_data_to_union], axis=1).drop_duplicates()

In [ ]:
joined_usda_cr_tracts.head()

In [ ]:
# joined_usda_cr_tracts.to_csv('joined_usda_cr_tracts.csv')